# Advanced programming: performance

## The rules of optimization


1. **Don't do it.**
2. **(For experts only) Don't do it yet.**

Inspired from famous internet programmer knowledge.

### Rules of thumb

<img src="https://imgs.xkcd.com/comics/is_it_worth_the_time_2x.png" alt="xkcd" width="400"/>

### If you really really want to do it anyway

1. **Write tests**
2. **Profile before optimizing**

Writing tests is important to keep the code running and making sure that it works the same way after optimizing. At the very least, keep a copy of the non-optimized code and test your optimized code against it.

Profiling helps to focus on what really matters. You might find the results of profiling very surprising!

## Some optimisation stories from the class

- vectorization instead of for-loops
- conversion between xarray/numpy arrays etc. is expensive - avoid it if you need performance
- writing functions instead of hard-coding every single step multiple times
- opening the same file every time in a for-loop instead of doing it once before starting the for-loop
- think of writing your own functions for routines you are using a lot instead of potentially slow package functions (e.g. `np.average`). See examples below.

## Profiling code

- default: https://docs.python.org/3/library/profile.html
- with graphics: https://github.com/benfred/py-spy

## Optimisation examples

```{important}

Always follow the further rules of scientific code optimisation in order:

1. Don't do it.
2. Don't do it (yet)
3. Test before doing it
4. Profile before doing it
5. Check vectorization (removing `for` loops)
6. Remove pandas / xarray overlay
7. Optimize numpy code
8. Check numba or PyPy 
8. Use multiprocessing
```

### Vectorization is awesome

This is ALWAYS what you should check first: can you replace a for loop with numpy only arithmetics?

In [82]:
import numpy as np

# Derivation naive
a = np.arange(12)**2
deriv = np.zeros(len(a))
for i in range(len(a)):
    if i == 0 or i == len(a) - 1:
        continue
    deriv[i] = (a[i+1] - a[i-1]) / 2
deriv

array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20.,  0.])

In [85]:
# Derivation vectorized
a = np.arange(12)**2
deriv = np.zeros(len(a))
deriv[1:-1] = (a[2:] - a[:-2]) / 2
deriv

array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20.,  0.])

Concrete example: salem and MetPy's `interp_1d` function (useful for interpolating model levels):
- [salem](https://github.com/fmaussion/salem/blob/0ae5885e4f466fc333a14544df851501f11102fd/salem/wrftools.py#L496-L551) uses naive interpolation but relies on multiprocessing to get speed-up
- [MetPy](https://github.com/Unidata/MetPy/blob/8b01e1fd22304ffcd5e933cfa901a229ab8fba3f/src/metpy/interpolate/one_dimension.py#L53-L173) uses clever vectorization and is faster.

### Back to basics: xarray -> numpy -> own functions 

Because of the "generalization overhead" (certain numpy and xarray functions are slow because they are meant to be general and do a lot of input checks.

If you know what your input looks like and have control over it, you can speed-up operations quite a bit.

#### xarray and pandas are slow on arithmetics 

In [86]:
import pandas as pd

In [87]:
df = pd.DataFrame()
df['data'] = np.ones(10000)

In [92]:
%timeit df**2

48.2 µs ± 4.77 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [93]:
%timeit df.values**2

7.45 µs ± 737 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [94]:
import xarray as xr
import numpy as np

da = xr.DataArray(np.random.uniform(size=(240, 300)))

In [95]:
%timeit da**2

50.3 µs ± 2.69 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [96]:
%timeit da.data**2

25.9 µs ± 878 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


*Question to class: why?*

#### numpy's general functions are sometimes slower than the direct arithmetic solution

In [97]:
d = np.arange(1, 1000)
w = d + 2

In [98]:
%timeit np.average(d, weights=w)

17.4 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [99]:
def my_avg(d, wgt):
    return (d * wgt).sum() / wgt.sum()

np.testing.assert_allclose(np.average(d, weights=w), my_avg(d, w))

In [100]:
%timeit my_avg(d, w)

4.23 µs ± 316 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


*Question to class: why?*

See also (similar story): https://github.com/numpy/numpy/issues/14281

**As for ALL cases of optimisation: please be aware of the risks! Make a careful gain / costs analysis.**

#### Array creation is costly 

In [101]:
%timeit np.ones(1000)

2.2 µs ± 401 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [102]:
%timeit np.empty(1000)

249 ns ± 16.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [106]:
dx = 100
nx = 200
surface_h = np.linspace(3000, 1000, nx)


def grad_np(surface_h, dx):
    """Numpy computing a gradient"""
    gradient = np.gradient(surface_h, dx)
    gradient[[-1, 0]] = 0
    return gradient


def my_grad(surface_h, dx):
    """Same result, not using numpy's function"""
    gradient = np.empty(surface_h.shape)
    gradient[1:nx-1] = (surface_h[2:] - surface_h[:nx-2])/(2*dx)
    gradient[[-1, 0]] = 0
    return gradient


def my_other_grad(surface_h, dx):
    """Same as above, but no array creation and using concatenate"""
    internal_grad = (surface_h[2:] - surface_h[:nx-2])/(2*dx)
    return np.concatenate([[0], internal_grad, [0]])

np.testing.assert_allclose(grad_np(surface_h, dx), my_grad(surface_h, dx))
np.testing.assert_allclose(grad_np(surface_h, dx), my_other_grad(surface_h, dx))

In [110]:
%timeit grad_np(surface_h, dx)

17.2 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [111]:
%timeit my_grad(surface_h, dx)

5.11 µs ± 332 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [112]:
%timeit my_other_grad(surface_h, dx)

4.99 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Can we make it even faster? Put array creation outside of the function (optimization only available when a for loop is involved):

In [114]:
gradient_container = np.zeros(surface_h.shape)

def my_super_grad(surface_h, dx):
    gradient_container[1:nx-1] = (surface_h[2:] - surface_h[:nx-2])/(2*dx)
    return gradient_container

np.testing.assert_allclose(grad_np(surface_h, dx), my_super_grad(surface_h, dx))

In [115]:
%timeit my_super_grad(surface_h, dx)

2.61 µs ± 97.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


#### But numpy is already super clever 

The following codes do the same. It is tempting to think that the second or third examples are faster (because they use in-place operations), but it turns out that they are all exactly the same speed (within errors):

In [116]:
def some_calc(surface_h):
    out = surface_h + 1
    out = surface_h * out
    out = out - 100
    out = out**2
    return out

In [117]:
def not_useful_optim(surface_h):
    return (surface_h * (surface_h + 1) - 100)**2

np.testing.assert_allclose(some_calc(surface_h), not_useful_optim(surface_h))

In [118]:
def also_not_useful_optim(surface_h):
    out = surface_h + 1
    out *= surface_h
    out -= 100
    return out**2

np.testing.assert_allclose(some_calc(surface_h), also_not_useful_optim(surface_h))

In [119]:
%timeit some_calc(surface_h)

3.51 µs ± 213 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [120]:
%timeit not_useful_optim(surface_h)

3.53 µs ± 138 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [121]:
%timeit also_not_useful_optim(surface_h)

3.61 µs ± 266 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


This is because python + numpy already knows how to compile this code to be optimized, i.e. it recognizes where the culprits are and solves them.

### Just in time compilation

For pure numpy code with unavoidable loops (typical for numerical models with a time stepping for example), [numba](https://numba.pydata.org) is probably the best shot you have at considerably speeding up your code. We'll make an example in class.

Numba is extensively used in some applications, e.g. for [cosipy](https://github.com/cryotools/cosipy). It has some drawbacks though (we discuss in class).

For some applications, [pypy](https://www.pypy.org) might be worth a shot but it requires using another python interpreter.